In [ ]:
import pandas as pd
import psycopg2
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
import numpy as np
import requests
from talib import MACD, RSI, BBANDS
from datetime import datetime, timedelta, date
from IPython.display import display, HTML
%matplotlib inline
import json
import math
from dateutil import parser
from pytz import timezone
import sys
import iso8601

In [ ]:
start_day_to_analyze = '2020-06-21'
end_day_to_analyze = '2020-06-22'
batch_id = "4c547d10-7789-4d00-a0de-b9fe3d3205f1"

In [ ]:
db_conn = create_engine("postgresql://momentum@localhost:5432/tradedb")

In [ ]:
api = tradeapi.REST(base_url="https://api.alpaca.markets")
session = requests.session()

In [ ]:
query = f'''
SELECT t.* 
FROM new_trades as t, algo_run as r
WHERE 
    t.algo_run_id = r.algo_run_id and
    r.batch_id = '{batch_id}' and
    tstamp >= '{start_day_to_analyze}' and 
    tstamp < '{end_day_to_analyze}' 
ORDER BY symbol, tstamp
'''
trades =  pd.read_sql_query(query, db_conn)
trades['client_time']= pd.to_datetime(trades['client_time'])

In [ ]:
query = f'''
SELECT * 
FROM 
algo_run as t
WHERE 
    start_time >= '{start_day_to_analyze}' and 
    start_time < '{end_day_to_analyze}' and
    batch_id = '{batch_id}'
    
ORDER BY algo_run_id
'''
algo_runs = pd.read_sql_query(query, db_conn)
algo_runs.set_index('algo_run_id', inplace=True)

In [ ]:
algo_runs

In [ ]:
trade_details = {}
batch = {}
for index, row in trades.iterrows():
    algo_run_id = row['algo_run_id']
    symbol = row['symbol']
    time = row['tstamp']
    operation = row['operation']
    price = row['price']
    indicators = row['indicators']
    qty = row['qty'] 
    stop_price = row['stop_price']
    target_price = row['target_price']
    if algo_run_id not in trade_details:
        trade_details[algo_run_id] = {}
    if symbol not in trade_details[algo_run_id]:
        trade_details[algo_run_id][symbol] = []
    trade_details[algo_run_id][symbol].append((operation, time, price, qty, indicators, stop_price, target_price))
    if batch_id not in batch:
        batch[batch_id] = []
    if algo_run_id not in batch[batch_id]:
        #print(batch_id, algo_run_id)
        batch[batch_id].append(algo_run_id)

In [ ]:
revenues = {}
how_was_my_day = {}

for batch_id, count in batch.items():
    revenues[batch_id] = []
    symbol_name = []
    counts = []
    for run_id in batch[batch_id]:
        
        symbols = trades.loc[trades['algo_run_id'] == run_id]['symbol'].value_counts()
        for symbol, count in symbols.items():
            revnue = 0
            for trade in trade_details[run_id][symbol]:
                delta = trade[2] * trade[3] * (1 if trade[0] == 'sell' or trade[0] == 'buy_short' else -1)
                revnue += delta

            revenues[batch_id].append(revnue)
            counts.append(count)
            symbol_name.append(symbol)


    d = {
        'symbol': symbol_name,
        'count': counts,
        'revenue': revenues[batch_id],
    }
    how_was_my_day[batch_id] = pd.DataFrame(data=d)

In [ ]:
for batch_id, how_did_i_do in how_was_my_day.items():
    #display(HTML(algo_runs.loc[algo_runs['batch_id'] == batch_id].to_html()))
    display(HTML(how_did_i_do.round(2).sort_values(by=['revenue']).to_html()))
    print(f'TOTAL REVENUE: {pd.Series(revenues[batch_id]).sum().round(2)} $ for {batch_id}')


In [ ]:
minute_history = {}
for batch_id, count in batch.items():
    for run_id in batch[batch_id]:
        symbols = trades.loc[trades['algo_run_id'] == run_id]['symbol'].value_counts()
        for symbol, count in symbols.items():
            if symbol not in minute_history:
                minute_history[symbol] = api.polygon.historic_agg_v2(
                    symbol, 
                    1, 
                    'minute',
                    _from = str((datetime.strptime(start_day_to_analyze, '%Y-%m-%d')-timedelta(days=10)).date()),
                    to=str((datetime.strptime(end_day_to_analyze, '%Y-%m-%d')+timedelta(days=1)).date())).df


In [ ]:
def grouper(iterable):
    prev = None
    group = []
    for item in iterable:
        
        if not prev or -0.02 <= float(item - prev) / prev <= 0.02:
            group.append(item)
        else:
            yield group
            group = [item]
        prev = item
    if group:
        yield group

def find_resistance(current_value, minute_history, now):
    """calculate next resistance"""
    now = pd.Timestamp(now) - timedelta(hours=4, minutes=1)
    #print(now)
    #print(minute_history)
    minute_history_index = minute_history["close"].index.get_loc(now, method='nearest')
    #print(f'index:{minute_history_index} data:{minute_history["close"][minute_history_index]}')
    #print(minute_history["close"][minute_history_index-10:minute_history_index])
    for back_track_min in range(120, len(minute_history.index), 60):
        #print("start")
        series = (
            minute_history["close"][-back_track_min:minute_history_index].dropna().between_time("9:30", "16:00").resample("5min").max()
        ).dropna()
        #print(series[-10:])
        # tlog(f"{symbol} find_resistances(): {len(series)}")

        diff = np.diff(series.values)
        #print(diff)
        high_index = np.where((diff[:-1] >= 0) & (diff[1:] <= 0))[0] + 1
        #print(high_index)
        #print(high_index)
        if len(high_index) > 0:
            #print(sorted(
            #    [series[i] for i in high_index if series[i]]
            #))
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] >= (current_value)]
            )
            #print(f"local_maximas={local_maximas}")
            if len(local_maximas) > 0:
                #print([i for i in high_index if series[i] >= current_value])
                return local_maximas
            continue
            
            clusters = dict(enumerate(grouper(local_maximas), 1))
            #print(f"cluster={clusters}")
            resistances = []
            for key, cluster in clusters.items():
                if len(cluster) > 1:
                    resistances.append(round(sum(cluster) / len(cluster), 2))
            resistances = sorted(resistances)

            if len(resistances) > 0:
                return resistances


    return None

def find_support(current_value, minute_history, now):
    """calculate support"""
    now = pd.Timestamp(now) - timedelta(hours=4)
    minute_history_index = minute_history["close"].index.get_loc(now, method='nearest')
    for back_track_min in range(120, len(minute_history.index), 60):
        series = (minute_history["close"][-back_track_min:minute_history_index].resample("5min").min()).dropna()
        diff = np.diff(series.values)
        high_index = np.where((diff[:-1] <= 0) & (diff[1:] > 0))[0] + 1
        if len(high_index) > 0:
            local_maximas = sorted(
                [series[i] for i in high_index if series[i] <= current_value]
            )
            if len(local_maximas) > 0:
                return local_maximas
            
    return None

In [ ]:
for symbol in minute_history:   
    symbol_df = trades.loc[trades['symbol']==symbol]
    start_date = symbol_df["client_time"].min()
    start_date = start_date.replace(hour=9, minute=30)
    print(start_date)
    cool_down_date = start_date + timedelta(minutes=5)
    minute_history_index = minute_history[symbol]["close"].index.get_loc(start_date, method='nearest')
    cool_minute_history_index = minute_history[symbol]["close"].index.get_loc(cool_down_date, method='nearest')
    open_price = minute_history[symbol]["close"][cool_minute_history_index]
    plt.plot(minute_history[symbol]["close"][minute_history_index:].between_time("9:30", "16:00"), label=symbol)
    plt.xticks(rotation=45)
 
    delta = 0
    profit = 0

    operations = []
    deltas = []
    profits = []
    times = []
    prices = []
    qtys = []
    indicators = []
    target_price = []
    stop_price = []
    resistances = []
    supports = []
    daily_change = []
    precent_vwap = []
    for index, row in symbol_df.iterrows(): 
        if row['operation'] == 'buy':
            try:
                now = int(row['client_time'])
                continue
            except Exception:
                pass 
            
            resistance = find_resistance(row['price'], minute_history[symbol], row['tstamp'])
            if resistance:
                for r in resistance:
                    plt.axhline(y=r, color="r")
            support = find_support(row['price'], minute_history[symbol], row['tstamp'])
            if support:
                for s in support:
                    plt.axhline(y=s, color="g")  

        delta = row['price'] * row['qty'] * (1 if row['operation'] == 'sell' or row['operation'] == 'buy_short' else -1)
        profit += delta
        plt.scatter(row['client_time'].to_pydatetime(), row['price'], c='g' if row['operation'] == 'buy' or row['operation'] == 'sell_short' else 'r', s=100)
        deltas.append(round(delta, 2))
        profits.append(round(profit, 2))
        operations.append(row['operation'])
        times.append(pd.to_datetime(row['client_time']))
        prices.append(row['price'])
        qtys.append(row['qty'])
        indicators.append(row['indicators'])
        target_price.append(row['target_price'])
        stop_price.append(row['stop_price'])    
        resistances.append(resistance)
        supports.append(support)
        daily_change.append(f"{round(100.0 * (row['price'] - open_price) / open_price, 2)}%")
        precent_vwap.append(f"{round(100.0 * (row['indicators']['avg'] - open_price) / open_price, 2)}%" if 'avg' in row['indicators'] else "")

    d = {
        'profit': profits,
        'trade': deltas,
        'operation': operations,
        'at': times,
        'price': prices,
        'support': supports,
        'resistance': resistances,
        'qty': qtys,
        'daily change': daily_change,
        'vwap': precent_vwap,
        'indicators': indicators,
        'target price': target_price,
        'stop price': stop_price
        
    }   
    print (f'{symbol} analysis with profit {round(profit, 2)}')
    display(HTML(pd.DataFrame(data=d).to_html()))
    plt.legend()
    plt.show()